# 06-04

- Transition Matrices
- BoxCox
- Centering and Scaling
- Euclidean Distance

In [29]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [30]:
cover = loadCoverGroups()
listCovers = cover[12]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [31]:
transitionMatrices = pd.read_csv('./data/unified/transitionMatrices.csv')
transitionMatrices = transitionMatrices.iloc[:, 1:]

In [32]:
transitionMatrices = transitionMatrices.sort_values('id')
transitionMatrices.id = transitionMatrices.id.astype('int64')
transitionMatrices = transitionMatrices.set_index('id')

In [33]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0


In [34]:
transitionMatrices.sum(axis = 1).sum(axis=0)

5979.999999999996

In [35]:
transitionMatrices['id'] = transitionMatrices.index

In [36]:
transitionMatrices['id']

id
23              23
105            105
118            118
120            120
292            292
            ...   
6002496    6002496
6002497    6002497
6002498    6002498
6002499    6002499
6002500    6002500
Name: id, Length: 5980, dtype: int64

In [37]:
transitionMatrices[transitionMatrices.id == originalSongId]

,0,1,2,3,4,5,6,7,8,9,...,1286,1287,1288,1290,1291,1292,1293,1294,1295,id
id,,,,,,,,,,,,,,,,,,,,,
6001918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6001918


In [38]:
transitionMatrices.reset_index(drop=True, inplace=True)

In [39]:
ids = transitionMatrices.id
df = transitionMatrices.iloc[:, 0:-1]

## Removing Skewness with boxcox

In [40]:
df = df + 1
cols = df.columns.tolist()

In [41]:
for colName in cols:
    df[colName] = stats.boxcox(df[colName])[0]

In [42]:
df

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.02919,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.008545,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.01840,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.01100,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.002244,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5976,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5977,0.00000,0.000737,0.0,0.000162,0.0,0.002546,0.008343,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5978,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.008594,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000121,0.0,0.0,0.0,0.000031,0.0,0.0


## Combining Centering and Scaling

In [43]:
cols = df.columns
scaler = StandardScaler()
scaler.fit(df)
data_normalized = scaler.transform(df)
print('mean: ', data_normalized.mean(axis=0).round(2))
print('std: ', data_normalized.std(axis=0).round(2))

df = pd.DataFrame(data_normalized, columns=cols)

mean:  [ 0. -0.  0. ...  0.  0.  0.]
std:  [1. 1. 1. ... 1. 1. 1.]


In [44]:
df

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
0,-0.647420,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
1,2.131096,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
2,-0.647420,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,2.729440,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
3,1.104025,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
4,0.399608,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,3.989592,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,-0.647420,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
5976,-0.647420,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,-0.399567,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
5977,-0.647420,6.374094,-0.060766,12.504736,-0.060766,3.870951,2.655684,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,-0.069808,-0.044841,-0.012933,-0.136892,-0.040927,-0.012933,-0.048442
5978,-0.647420,-0.157063,-0.060766,-0.079970,-0.060766,-0.259785,2.747334,-0.251885,-0.048442,-0.094563,...,-0.028928,-0.051795,-0.102355,14.325044,-0.044841,-0.012933,-0.136892,24.433583,-0.012933,-0.048442


## Applying eucledian distance

In [45]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [46]:
originalSongIndex = ids[ids == originalSongId].index[0]
originalSongIndex

originalSongVector = df.iloc[originalSongIndex, :]

***Dropping the cover song***

In [47]:
coverIndex = ids[ids == coverSongId].index[0]
coverSongVector = df.iloc[coverIndex, :]

In [48]:
ranking = []
for index in range(0, len(df)):
    ranking.append([ids[index], euclidean_distance(df.iloc[index, :], originalSongVector)])

In [49]:
ranksDF = pd.DataFrame(ranking)

In [50]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [51]:
ranksDF

,id,rank
0,23,15.568688
1,105,12.914058
2,118,21.566799
3,120,18.132856
4,292,17.279933
...,...,...
5975,6002496,18.523962
5976,6002497,13.520362
5977,6002498,107.220342
5978,6002499,64.259223


In [52]:
ranksDF = ranksDF.sort_values(by="rank")

In [53]:
ranksDF.reset_index(drop=True, inplace=True)

In [54]:
ranksDF.head(10)

,id,rank
0,6001918,0.000000
1,6001674,8.424217
2,6001758,8.555779
3,6001673,9.288070
4,6001761,9.288070
5,6001670,9.752545
6,6000424,9.813484
7,6001624,10.031573
8,49158,10.111099
9,6001853,10.399804


In [55]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
5561,6001997,57.61747


## Rank for original Song 450

In [56]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,6001918,0.00000
5561,6001997,57.61747
